### 1 - Import test data

In [14]:
pip install oandapyV20 apscheduler



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/51.6 kB ? eta -:--:--
     ---------------------- --------------- 30.7/51.6 kB 262.6 kB/s eta 0:00:01
     -------------------------------------  51.2/51.6 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 51.6/51.6 kB 240.6 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for apscheduler from https://files.pythonhosted.org/packages/13/b5/7af0cb920a476dccd612fbc9a21a3745fb29b1fcd74636078db8f7ba294c/APScheduler-3.10.4-py3-none-any.whl.metadata
  Obtaining dependency information for tzlocal!=3.*,>=2.0 from https://files

In [5]:
pip install yfinance



  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/51/31/72b8f5aa9ed9c4a6afd09c0bab491862ba5837facf7d81e1ed51a555ae8e/yfinance-0.2.44-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pandas>=1.3.0 from https://files.pythonhosted.org/packages/29/d4/1244ab8edf173a10fd601f7e13b9566c1b525c4f365d6bee918e68381889/pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.16.5 from https://files.pythonhosted.org/packages/b7/98/5640a09daa3abf0caeaefa6e7bf0d10c0aa28a77c84e507d6a716e0e23df/numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.7 kB 660.6 kB/s eta 0:00:01
     -------------------------------- ----- 51.2/59.7 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 59.7/59.7 kB 526.6 kB/s eta 0:00:00
  Obtaining dependency information for requests>=2.31 fr

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-09-01", end="2024-09-30", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-09-02 00:00:00,1.104972,1.104972,1.104850,1.104972,1.104972,0
2024-09-02 00:15:00,1.104972,1.104972,1.104850,1.104850,1.104850,0
2024-09-02 00:30:00,1.104972,1.104972,1.104728,1.104728,1.104728,0
2024-09-02 00:45:00,1.104850,1.104850,1.104728,1.104728,1.104728,0
2024-09-02 01:00:00,1.104728,1.104850,1.104484,1.104850,1.104850,0
...,...,...,...,...,...,...
2024-09-27 21:15:00,1.116695,1.117069,1.116570,1.117069,1.117069,0
2024-09-27 21:30:00,1.116944,1.116944,1.116570,1.116570,1.116570,0
2024-09-27 21:45:00,1.116570,1.116695,1.116196,1.116570,1.116570,0


### 2 - Define your signal function

In [31]:
# uses the last two periods of price data to identify bearish and bullish trading patterns 
# (1 for bearish, 2 for bullish, and 0 for no signal)
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [32]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    1764
2      69
1      63
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [17]:
import os

In [33]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [35]:
def get_candles(n):
    # Using the provided access token directly
    access_token = '50724f762b0cddbfff5b6858fd487cb3-28bdfa02d7666361915467baa618411e'
    
    # Initialize the OANDA API client
    client = API(access_token=access_token, environment='practice')
    
    # Set up the parameters for the candle request
    params = {
        "granularity": "M15",  # 15-minute candles
        "count": n,            # Fetch 'n' candles
        "price": "B",          # 'B' for bid prices
    }
    
    # Request candles for EUR/USD
    candles_request = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
    
    # Make the request and return the response
    response = client.request(candles_request)
    
    # Extract the candles
    candles = response['candles']
    return candles

# Fetch 3 candles
candles = get_candles(3)

# Process and print the bid open prices for the candles
for candle in candles:
    # 'candle' is a dictionary containing time, open, high, low, close, etc.
    bid_open = float(candle['bid']['o'])  # 'o' is the open price for the bid
    print(bid_open > 1)

True
True
True


In [36]:
def trading_job():
    # Fetch candles using the modified get_candles function
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', 'Low'])
    
    # Populate the DataFrame with candle data
    for i, candle in enumerate(candles):
        dfstream.loc[i, 'Open'] = float(candle['bid']['o'])  # Open price
        dfstream.loc[i, 'Close'] = float(candle['bid']['c'])  # Close price
        dfstream.loc[i, 'High'] = float(candle['bid']['h'])   # High price
        dfstream.loc[i, 'Low'] = float(candle['bid']['l'])    # Low price

    # Ensure the DataFrame columns are of type float
    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    # Signal generator (assumed to be defined elsewhere)
    signal = signal_generator(dfstream.iloc[:-1, :])

    # EXECUTING ORDERS
    accountID = '001-001-12614365-001'
    access_token = '50724f762b0cddbfff5b6858fd487cb3-28bdfa02d7666361915467baa618411e'
    client = API(access_token)

    SLTPRatio = 2.0
    previous_candleR = abs(dfstream['High'].iloc[-2] - dfstream['Low'].iloc[-2])

    # Set up Stop Loss and Take Profit prices
    SLBuy = dfstream['Open'].iloc[-1] - previous_candleR
    SLSell = dfstream['Open'].iloc[-1] + previous_candleR
    TPBuy = dfstream['Open'].iloc[-1] + previous_candleR * SLTPRatio
    TPSell = dfstream['Open'].iloc[-1] - previous_candleR * SLTPRatio

    print(dfstream.iloc[:-1, :])  # Print the DataFrame of previous candles
    print(TPBuy, SLBuy, TPSell, SLSell)  # Print TP and SL values

    # Trading signal (1 for sell, 2 for buy)
    if signal == 1:
        mo = MarketOrderRequest(
            instrument="EUR_USD",
            units=-1000,
            takeProfitOnFill=TakeProfitDetails(price=TPSell).data,
            stopLossOnFill=StopLossDetails(price=SLSell).data
        )
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

    elif signal == 2:
        mo = MarketOrderRequest(
            instrument="EUR_USD",
            units=1000,
            takeProfitOnFill=TakeProfitDetails(price=TPBuy).data,
            stopLossOnFill=StopLossDetails(price=SLBuy).data
        )
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [38]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

      Open    Close     High      Low
0  1.10633  1.10658  1.10669  1.10626
1  1.10659  1.10679  1.10681  1.10632
1.1077500000000002 1.10628 1.1057899999999998 1.1072600000000001
